In [1]:
#Import libraries
os.chdir("/home/robbe/ionbot/")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
import numpy as np
import os
from pathlib import Path
from pyteomics import mass as pymass
import OxiAnalysis as OA
import importlib
importlib.reload(OA)
import warnings; warnings.simplefilter('ignore')
import uniprot
import pprint
import sys
import urllib

In [2]:
master = pd.read_csv("/home/robbe/ionbot/mastersets/master_first.csv")
masterpr = pd.read_csv("/home/robbe/ionbot/mastersets/PXD012477_first_proteins.csv")
mastermod = pd.read_csv("/home/robbe/ionbot/mastersets/PXD012477_modifications.csv")

In [3]:
#PXD006657 was still included in masterset but won't be used
master = master[master["Accession"] != 6657]
masterpr = masterpr[masterpr["Accession"] != 6657]
mastermod = mastermod[mastermod["Accession"] != 6657]

In [4]:
g = mastermod.groupby(["protein", "uniprot_id", "unexpected_modification", "position"])["#PSMs"].sum().to_frame().reset_index()
g

,protein,uniprot_id,unexpected_modification,position,#PSMs
0,1433B_HUMAN,P31946,[34]Methyl[N-TERM],63,1
1,1433B_HUMAN,P31946,[35]Oxidation[S],47,2
2,1433B_HUMAN,P31946,[35]Oxidation[Y],50,25
3,1433B_HUMAN,P31946,[35]oxidation[M],220,120
4,1433B_HUMAN,P31946,[4]Carbamidomethyl[Y],50,1
...,...,...,...,...,...
97997,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[99998]ICPL_2H(4)[K],258,1
97998,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[999992]GIST-Quat_2H(3)[N-TERM],279,3
97999,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[999993]GIST-Quat_2H(3)[K],258,13
98000,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[999999]Galactosyl[K],258,2


In [5]:
g10 = g[g["#PSMs"] > 10]

In [6]:
g10[g10["unexpected_modification"].isin(OA.modslist)]

,protein,uniprot_id,unexpected_modification,position,#PSMs
2,1433B_HUMAN,P31946,[35]Oxidation[Y],50,25
3,1433B_HUMAN,P31946,[35]oxidation[M],220,120
14,1433E_HUMAN,P62258,[35]Oxidation[Y],49,25
16,1433E_HUMAN,P62258,[35]oxidation[M],221,120
32,1433F_HUMAN,Q04917,[35]Oxidation[Y],49,25
...,...,...,...,...,...
97774,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[W],147,35
97775,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[W],283,52
97776,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[W],479,21
97777,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[Y],155,14


Goal: per modification have a list of proteins, find out accessible surface of the modified residue, collect in a list

In [7]:
moddf = g10[g10["unexpected_modification"] == "[35]Oxidation[Y]"]
moddf[["uniprot_id", "position"]]

,uniprot_id,position
2,P31946,50
14,P62258,49
32,Q04917,49
40,P61981,49
54,P31947,48
...,...,...
97150,sp|TRFE_HUMAN|,536
97151,sp|TRFE_HUMAN|,578
97152,sp|TRFE_HUMAN|,593
97153,sp|TRFE_HUMAN|,669


In [8]:
moddict = dict()
for mod in OA.modslist:
    moddf = g10[g10["unexpected_modification"] == mod]
    #Remove contaminants
    moddf = moddf[~moddf["uniprot_id"].str.startswith("sp|")]
    protein_list = moddf["uniprot_id"].tolist()
    poslist = moddf["position"].tolist()
    pro_poslist = []
    for i in range(len(protein_list)):
        pro_pos = protein_list[i] + "|" + str(poslist[i])
        pro_poslist.append(pro_pos)
    moddict[mod] = pro_poslist

In [9]:
pro_poslist

['P31946|220',
 'P62258|221',
 'Q04917|223',
 'P61981|223',
 'P31947|155',
 'P31947|220',
 'P27348|218',
 'P63104|218',
 'P02763|129',
 'P19652|129',
 'P19652|174',
 'P20848|89',
 'P01009|250',
 'P01009|266',
 'P01009|375',
 'P01009|382',
 'P01009|398',
 'P01009|409',
 'P04217|123',
 'P01023|151',
 'P01023|666',
 'P01023|688',
 'P01023|713',
 'P01023|1314',
 'P01023|1378',
 'P01023|1385',
 'A8K2U0|144',
 'A8K2U0|421',
 'A8K2U0|1116',
 'A8K2U0|1364',
 'A8K2U0|1378',
 'P05067|303',
 'P01011|204',
 'P01011|205',
 'P01011|219',
 'P01011|243',
 'P01011|284',
 'P01011|290',
 'P01011|401',
 'P01011|414',
 'O94911|831',
 'Q9P1F3|1',
 'Q6UWZ7|187',
 'P07108|25',
 'P07108|47',
 'Q4L235|486',
 'P62736|46',
 'P62736|49',
 'P62736|84',
 'P62736|192',
 'P62736|327',
 'Q562R1|45',
 'Q562R1|48',
 'Q562R1|154',
 'Q562R1|191',
 'P60709|44',
 'P60709|47',
 'P60709|82',
 'P60709|153',
 'P60709|190',
 'P60709|305',
 'P60709|325',
 'P68032|46',
 'P68032|49',
 'P68032|84',
 'P68032|192',
 'P68032|327',
 'P63

In [275]:
from pypdb import *
found_pdbs = Query("P19652").search()
found_pdbs[0]

'3APU'

In [12]:
def download_pdb(pdbcode, datadir, downloadurl="https://files.rcsb.org/download/"):
    """
    Downloads a PDB file from the Internet and saves it in a data directory.
    :param pdbcode: The standard PDB ID e.g. '3ICB' or '3icb'
    :param datadir: The directory where the downloaded file will be saved
    :param downloadurl: The base PDB download URL, cf.
        `https://www.rcsb.org/pages/download/http#structures` for details
    :return: the full path to the downloaded PDB file or None if something went wrong
    """
    pdbfn = pdbcode + ".pdb"
    url = downloadurl + pdbfn
    outfnm = os.path.join(datadir, pdbfn)
    try:
        urllib.request.urlretrieve(url, outfnm)
        return outfnm
    except Exception as err:
        print(str(err), file=sys.stderr)
        return None

In [276]:
from Bio.PDB import PDBParser
import requests as r
from Bio import SeqIO
from io import StringIO
from pypdb import *
d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}
pdb_file = download_pdb(found_pdbs[0], '/home/robbe/ionbot/PDB_files')


# run parser
parser = PDBParser(QUIET=True)
structure = parser.get_structure('struct', pdb_file)    

# iterate each model, chain, and residue
# printing out the sequence for each chain

for model in structure:
    PDBseqlist = []
    for chain in model:
        
        for residue in chain:
            if residue.resname in d3to1:
                PDBseqlist.append(d3to1[residue.resname])
                PDBseq = "".join(PDBseqlist)


In [277]:
print(found_pdbs[0])

3APU


In [278]:
print(PDBseq)


QIPLCANLVPVPITNATLDRITGKWFYIASAFRNEEYNKSVQEIQATFFYFTPNKTEDTIFLREYQTRQNQCFYNSSYLNVQRENGTVSRYEGGREHVAHLLFLRDTKTLMFGSYLDDEKNWGLSFYADKPETTKEQLGEFYEALDCLRIPRSDVMYTDWKKDKCEPLEKQHEKEIPLCANLVPVPITNATLDRITGKWFYIASAFRNEEYNKSVQEIQATFFYFTPNKTEDTIFLREYQTRQNQCFYNSSYLNVQRENGTVSRYEGGREHVAHLLFLRDTKTLMFGSYLDDEKNWGLSFYADKPETTKEQLGEFYEALDCLRIPRSDVMYTDWKKDKCEPLEKQHE


In [279]:

# for i in pro_poslist:
cID= 'P19652'                  #i.split("|")[0]

baseUrl="http://www.uniprot.org/uniprot/"
currentUrl=baseUrl+cID+".fasta"
response = r.post(currentUrl)
cData=''.join(response.text)

Seq=StringIO(cData)
pSeq=SeqIO.parse(Seq,'fasta')
pdb_found = Query(cID).search()[0]
for record in pSeq:
    Uniprotseq = record.seq
    
        


In [280]:
pdb_found

'3APU'

In [281]:
print(Uniprotseq)

MALSWVLTVLSLLPLLEAQIPLCANLVPVPITNATLDRITGKWFYIASAFRNEEYNKSVQEIQATFFYFTPNKTEDTIFLREYQTRQNQCFYNSSYLNVQRENGTVSRYEGGREHVAHLLFLRDTKTLMFGSYLDDEKNWGLSFYADKPETTKEQLGEFYEALDCLCIPRSDVMYTDWKKDKCEPLEKQHEKERKQEEGES


In [293]:
from Bio import pairwise2
from Bio.Seq import Seq
from Bio.pairwise2 import format_alignment
alignments = pairwise2.align.globalxx(Uniprotseq, PDBseq)
print(format_alignment(*alignments[0]))

M-----A-LS----------------W---------------V--------------------L----T-------------V-------------------L------------S-L-------L------PL------L-E---A--------------------------Q----IPLCANLVPVPITNATLDRITGKWFYIASAFRNEEYNKSVQEIQATFFYFTPNKTEDTIFLREYQTRQNQCFYNSSYLNVQRENGTVSRYEGGREHVAHLLFLRDTKTLMFGSYLDDEKNWGLSFYADKPETTKEQLGEFYEALDCLC-IPRSDVMYTDWKKDKCEPLEKQHEKERKQEEGES
      | |                 |               |                    |    |             |                   |            | |       |      |       | |   |                          |    |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||  |||||||||||||||||||||||         | 
-QIPLCANL-VPVPITNATLDRITGKWFYIASAFRNEEYNKSVQEIQATFFYFTPNKTEDTIFLREYQTRQNQCFYNSSYLNVQRENGTVSRYEGGREHVAHLLFLRDTKTLMFGSYLDDEKNWGLSFYADKP-ETTKEQLGEFYEALDCLRIPRSDVMYTDWKKDKCEPLEKQHEKEIPLCANLVPVPITNATLDRITGKWFYIASAFRNEEYNKSVQEIQATFFYFTPNKTEDTIFLREYQTRQNQCFYNSSYLNVQRENGTVSRYEGGREHVA

In [288]:
string = "-QIPLCANL-VPVPITNATLDRITGKWFYIASAFRNEEYNKSVQEIQATFFYFTPNKTEDTIFLREYQTRQNQCFYNSSYLNVQRENGTVSRYEGGREHVAHLLFLRDTKTLMFGSYLDDEKNWGLSFYADKP-ETTKEQLGEFYEALDCLRIPRSDVMYTDWKKDKCEPLEKQHEKEIPLCANLVPVPITNATLDRITGKWFYIASAFRNEEYNKSVQEIQATFFYFTPNKTEDTIFLREYQTRQNQCFYNSSYLNVQRENGTVSRYEGGREHVAHLLFLRDTKTLMFGSYLDDEKNWGLSFYADKPETTKEQLGEFYEALDCL-RIPRSDVMYTDWKKDKCEPLEKQH---------E-"
string.replace("-","")[320:330]

'CLRIPRSDVM'

In [294]:
from Bio import pairwise2

def seqmatch(alignment, pos): #TODO do sanity check met if nieuw gemapte AA == doel AA of check nog even hoe beter op te lossen
    pos = pos - 1
    matches = []

    for i, (a, b) in enumerate(zip(alignment[0], alignment[1])):
        try:
            if alignment.seqA[pos] == alignment.seqB[pos]:
                PDBseq2B = alignment.seqB[0:pos+1]
                countB = PDBseq2B.count("-")
                PDBseq2A = alignment.seqA[0:pos+1]
                countA = PDBseq2A.count("-")
                PDBseq2 = alignment.seqB.replace('-', "")
                if countA > countB:
                    need_added = countA - countB
                    PDBpos = pos + need_added
                elif countA < countB:
                    need_substracted = countB - countA
                    PDBpos = pos - need_substracted
                else:
                    PDBpos = pos
                #PDBpos = pos - count #get index of the AA out of this
                return PDBpos
            else:
                return None
        except IndexError:
                return None
        #     return True
        # else:
        #     return False

m = seqmatch(alignments[0], 174)
print(m)

325


In [295]:
print(Uniprotseq[173])

M


In [296]:
print(PDBseq[325])

R


In [182]:
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
model = structure[0]
dssp = DSSP(model, pdb_file)

In [183]:
a_key = list(dssp)
for i in a_key:
    if i[1] == "X":
        a_key.remove(i)

In [184]:
test = []
for i in a_key:
    test.append(i[1])

In [191]:
test[218]

'M'

In [185]:
DSSPseqnoX = "".join(test)

In [187]:
a_key[218][1]

'M'

In [273]:
from pypdb import *
from Bio.PDB import PDBParser
import requests as r
from Bio import SeqIO
from io import StringIO
from Bio import pairwise2
from Bio.Seq import Seq
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
check_list = []
for i in pro_poslist:
    protein = i.split("|")[0]
    pos = int(i.split("|")[1])
    found_pdbs = Query(protein).search()

    baseUrl="http://www.uniprot.org/uniprot/"
    currentUrl=baseUrl+protein+".fasta"
    response = r.post(currentUrl)
    cData=''.join(response.text)

    Seq=StringIO(cData)
    pSeq=SeqIO.parse(Seq,'fasta')
    
    for record in pSeq:
        Uniprotseq = record.seq
    #first search result in PDB for Uniprot Accession
    if found_pdbs != None:
        PDB_ID = found_pdbs[0]
        d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
        'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
        'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
        'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}
        pdb_file = download_pdb(PDB_ID, '/home/robbe/ionbot/PDB_files')


        # run parser
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure('struct', pdb_file)    

        # iterate each model, chain, and residue
        # printing out the sequence for each chain

        for model in structure:
            PDBseqlist = []
            for chain in model:
                
                for residue in chain:
                    if residue.resname in d3to1:
                        PDBseqlist.append(d3to1[residue.resname])
                        PDBseq = "".join(PDBseqlist)
        
        
        alignments = pairwise2.align.globalxx(Uniprotseq, PDBseq)
        m = seqmatch(alignments[0], pos)
        model = structure[0]
        try:
            dssp = DSSP(model, pdb_file)
        
            a_key = list(dssp)
            for i in a_key:
                if i[1] == "X":
                    a_key.remove(i)
            if m != None:
                RSA = a_key[m][1]
                check_list.append("{}|{}==>{}:{}".format(protein, pos +1, m,  RSA))
        except Exception:
            continue

KeyboardInterrupt: 

In [274]:
check_list

['P62258|222==>218:M',
 'P31947|156==>159:M',
 'P31947|221==>224:M',
 'P27348|219==>217:M',
 'P63104|219==>214:M',
 'P02763|130==>109:M',
 'P19652|175==>325:R',
 'P01009|251==>206:M',
 'P01009|267==>222:M',
 'P01009|376==>331:M',
 'P01009|399==>354:M',
 'P01009|410==>365:M',
 'P01023|1379==>41:A',
 'P01023|1386==>48:V',
 'A8K2U0|145==>124:M',
 'A8K2U0|422==>388:M',
 'A8K2U0|1117==>1013:M',
 'A8K2U0|1365==>1253:M',
 'A8K2U0|1379==>1267:M',
 'P01011|402==>709:L',
 'P01011|415==>722:Q',
 'Q9P1F3|2==>0:M',
 'P60709|45==>42:M',
 'P60709|48==>45:M',
 'P60709|83==>80:M',
 'P60709|154==>151:M',
 'P60709|191==>188:M',
 'P60709|306==>303:M',
 'P60709|326==>323:M']